# 03. Learning LangGraph - Agent Supervisor

In [ ]:
!pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "LangGraph 03 Multi-agent Supervisor"

In [1]:
from agents.utils.utils import init

init()

## The model

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview")

## Tools

In [3]:
from typing import Annotated, List, Tuple, Union
from langchain.tools import BaseTool, StructuredTool, Tool
from langchain_experimental.tools import PythonREPLTool
from langchain_core.tools import tool
import random

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

In [4]:
# my custom tools
@tool("lower_case", return_direct=False)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=False)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100. input the word 'random'"""
    return random.randint(0, 100)

tools = [to_lower_case,random_number_maker,python_repl_tool]

## Helper Utils

In [5]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [6]:
# agent node
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

## Create Agent Supervisor

It will use function calling to choose the next worker node OR finish processing.

In [7]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Lotto_Manager", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))


supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

/var/folders/fx/hq7s2yzs0cj06n4k_1p8hm9w0000gn/T/ipykernel_45805/2464156963.py:48: LangChainDeprecationWarning: The method `BaseChatOpenAI.bind_functions` was deprecated in langchain-openai 0.2.1 and will be removed in 1.0.0. Use :meth:`~langchain_openai.chat_models.base.ChatOpenAI.bind_tools` instead.
  | llm.bind_functions(functions=[function_def], function_call="route")


## Create the Agnet State and Graph

In [9]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


lotto_agent = create_agent(llm, tools, "You are a senior lotto manager. you run the lotto and get random numbers")
lotto_node = functools.partial(agent_node, agent=lotto_agent, name="Lotto_Manager")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_agent(llm, [python_repl_tool], "You may generate safe python code to analyze data and generate charts using matplotlib.")
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Lotto_Manager", lotto_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)

### Edges

In [10]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor") # add one edge for each of the agents

# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

Run it

In [11]:
config = {"recursion_limit": 20}
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Get 10 random lotto numbers and plot them on a histogram in 10 bins and tell me what the 10 numbers are at the end")
        ]
    }, config=config
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Lotto_Manager'}}
----


Python REPL can execute arbitrary code. Use with caution.


{'Lotto_Manager': {'messages': [HumanMessage(content="It appears that the `matplotlib` library is not available in the current environment, so I'm unable to plot the histogram for you. However, I can still provide you with the 10 random lotto numbers that were generated. Here are the numbers:\n\n1. 80\n2. 61\n3. 47\n4. 33\n5. 14\n6. 48\n7. 85\n8. 84\n9. 64\n10. 18\n\nEach number is unique and falls within the range of 0 to 100, as per typical lotto number draws. You can use these numbers for your lotto purposes. If you need assistance with anything else, please let me know!", additional_kwargs={}, response_metadata={}, name='Lotto_Manager')]}}
----
{'supervisor': {'next': 'Coder'}}
----
{'Coder': {'messages': [HumanMessage(content="I apologize for the confusion earlier. It looks like we don't have the `matplotlib` library available to plot the histogram directly. However, I was able to generate 10 random lotto numbers for you.\n\nHere are the 10 random lotto numbers that were generated

In [12]:
final_response = graph.invoke(
    {
        "messages": [
            HumanMessage(content="Get 10 random lotto numbers and plot them on a histogram in 10 bins and tell me what the 10 numbers are at the end")
        ]
    }, config=config
)

In [ ]:
final_response

{'messages': [HumanMessage(content='Get 10 random lotto numbers and plot them on a histogram in 10 bins and tell me what the 10 numbers are at the end'),
  HumanMessage(content='The histogram has been plotted using the following lotto numbers:\n\n- 70\n- 75\n- 37\n- 19\n- 38\n- 29\n- 34\n- 50\n- 22\n- 19\n\nUnfortunately, I cannot directly display images here, but the histogram was created with 10 bins, each representing a range of numbers, and the frequency of the numbers falling into each bin is depicted. You can recreate the histogram using any software that supports plotting, such as Python with Matplotlib, using the numbers provided above.', name='Lotto_Manager')],
 'next': 'FINISH'}

In [ ]:
final_response['messages'][1].content

'The histogram has been plotted using the following lotto numbers:\n\n- 70\n- 75\n- 37\n- 19\n- 38\n- 29\n- 34\n- 50\n- 22\n- 19\n\nUnfortunately, I cannot directly display images here, but the histogram was created with 10 bins, each representing a range of numbers, and the frequency of the numbers falling into each bin is depicted. You can recreate the histogram using any software that supports plotting, such as Python with Matplotlib, using the numbers provided above.'